In [9]:
def run_pandas():
    """Implementação completa com Pandas"""

    import pandas as pd
    import numpy as np
    from sklearn.preprocessing import LabelEncoder, StandardScaler
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.impute import KNNImputer

    #---------------------------------------------------------------------------------------------
    # Parte 1: Modelo de ML (ML_Pisa) completa os valores omissos do dataset pisa_2006-2018
    #---------------------------------------------------------------------------------------------

    pisa = pd.read_csv("pisa_2006-2018.csv")
    pisa.replace('NA', np.nan, inplace=True)
    pisa['Score'] = pd.to_numeric(pisa['Score'], errors='coerce')

    # Codificação
    le = LabelEncoder()
    pisa['Country_encoded'] = le.fit_transform(pisa['Country'])
    pisa['Subject_encoded'] = le.fit_transform(pisa['Subject'])

    # Modelo
    train = pisa.dropna(subset=['Score'])
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(train[['Year', 'Subject_encoded', 'Country_encoded']], train['Score'])

    # Previsão
    missing = pisa[pisa['Score'].isna()]
    if not missing.empty:
        pred = model.predict(missing[['Year', 'Subject_encoded', 'Country_encoded']])
        pisa.loc[pisa['Score'].isna(), 'Score'] = np.round(pred)

    pisa.to_csv("pisa_imputed_pandas.csv", index=False)


    #---------------------------------------------------------------------------------------------
    # Parte 2: Junção dos datasets
    #---------------------------------------------------------------------------------------------

    # Carregar os datasets
    pisa_imputed = pd.read_csv("pisa_imputed_pandas.csv")
    gdp = pd.read_csv("GDP.csv")

    # Corrigir possíveis espaços nos nomes das colunas
    gdp.columns = gdp.columns.str.strip()

    # Filtrar apenas os anos de interesse no dataset PISA
    pisa_imputed = pisa_imputed[pisa_imputed['Year'].isin([2012, 2015, 2018])]

    # Filtrar colunas relevantes do GDP
    gdp = gdp[['Country', 'Country Code', '2012', '2015', '2018']]

    # Obter lista de países comuns
    paises_comuns = set(pisa_imputed['Country']).intersection(set(gdp['Country']))

    # Filtrar os datasets para manter apenas países comuns
    gdp_filtrado = gdp[gdp['Country'].isin(paises_comuns)].copy()
    pisa_filtrado = pisa_imputed[pisa_imputed['Country'].isin(paises_comuns)].copy()

    # Transformar GDP de wide para long
    gdp_long = gdp_filtrado.melt(
        id_vars=['Country', 'Country Code'],
        value_vars=['2012', '2015', '2018'],
        var_name='Year',
        value_name='GDP'
    )

    # Garantir que o ano está como inteiro
    gdp_long['Year'] = gdp_long['Year'].astype(int)

    # Transformar PISA de long para wide (Subject → colunas)
    pisa_wide = pisa_filtrado[['Country', 'Year', 'Subject', 'Score']].pivot_table(
        index=['Country', 'Year'],
        columns='Subject',
        values='Score'
    ).reset_index()

    # Juntar os datasets
    dados_combinados = pd.merge(gdp_long, pisa_wide, on=['Country', 'Year'], how='left')

    # Reestruturar para wide por ano
    pisa_e_GDP = dados_combinados.pivot(
        index=['Country', 'Country Code'],
        columns='Year',
        values=['GDP', 'Maths', 'Science', 'Reading']
    )

    # Ajustar nomes das colunas para formato mais legível
    pisa_e_GDP.columns = [f"{var}_{year}" for var, year in pisa_e_GDP.columns]
    pisa_e_GDP = pisa_e_GDP.reset_index()
    pisa_e_GDP = pisa_e_GDP.rename(columns={
        'Country Code': 'Country_code'
    })

    pisa_e_GDP.to_csv("pisa_e_GDP.csv", index=False)


    #---------------------------------------------------------------------------------------------
    # Parte 3: Processamento do GDP_PISA (ML_GDP_Pisa)
    #---------------------------------------------------------------------------------------------

    dt = pd.read_csv("pisa_e_GDP.csv")

    # Tratamento de NAs
    print("\n[Pandas] Valores NaN antes:", dt.isna().sum().sum())

    # Salvar máscara de valores ausentes para usar depois
    missing_mask = dt.isna()

    colunas_numericas = dt.select_dtypes(include=[np.number]).columns
    scaler = StandardScaler()
    dt_scaled = dt.copy()
    dt_scaled[colunas_numericas] = scaler.fit_transform(dt[colunas_numericas])

    imputer = KNNImputer(n_neighbors=5)
    dt_imputed_scaled = dt_scaled.copy()
    dt_imputed_scaled[colunas_numericas] = imputer.fit_transform(dt_scaled[colunas_numericas])

    dt_imputed = dt.copy()
    dt_imputed[colunas_numericas] = scaler.inverse_transform(dt_imputed_scaled[colunas_numericas])

    # Formatação
    colunas_gdp = ["GDP_2012", "GDP_2015", "GDP_2018"]
    dt_imputed[colunas_gdp] = dt_imputed[colunas_gdp].round(5)

    # Correção: Aplicar arredondamento e conversão para int apenas nas células que eram NaN
    # Exclui colunas GDP que já foram tratadas separadamente
    colunas_para_arredondar = [col for col in colunas_numericas if col not in colunas_gdp]

    for col in colunas_para_arredondar:
        if missing_mask[col].any():
            dt_imputed.loc[missing_mask[col], col] = dt_imputed.loc[missing_mask[col], col].round(0).astype(int)

    print("[Pandas] Valores NaN após:", dt_imputed.isna().sum().sum())


    #---------------------------------------------------------------------------------------------
    # Parte 4: Adicionar as médias dos anos e das disciplinas
    #---------------------------------------------------------------------------------------------

    # Criar novas colunas com médias das disciplinas arredondadas a 3 casas decimais
    dt_imputed['Means_Maths'] = dt_imputed[['Maths_2012', 'Maths_2015', 'Maths_2018']].mean(axis=1, skipna=True).round(3)
    dt_imputed['Means_Science'] = dt_imputed[['Science_2012', 'Science_2015', 'Science_2018']].mean(axis=1, skipna=True).round(3)
    dt_imputed['Means_Reading'] = dt_imputed[['Reading_2012', 'Reading_2015', 'Reading_2018']].mean(axis=1, skipna=True).round(3)

    dt_imputed['Mean_2012'] = dt_imputed[[c for c in dt_imputed.columns if '2012' in c and 'GDP' not in c]].mean(axis=1).round(3)
    dt_imputed['Mean_2015'] = dt_imputed[[c for c in dt_imputed.columns if '2015' in c and 'GDP' not in c]].mean(axis=1).round(3)
    dt_imputed['Mean_2018'] = dt_imputed[[c for c in dt_imputed.columns if '2018' in c and 'GDP' not in c]].mean(axis=1).round(3)


    #---------------------------------------------------------------------------------------------
    # Parte 5: Adicionar variações relativas (%) entre anos
    #---------------------------------------------------------------------------------------------

    # Variação percentual do GDP
    dt_imputed['GDP_change_2012_2015'] = ((dt_imputed['GDP_2015'] - dt_imputed['GDP_2012']) / dt_imputed['GDP_2012'] * 100).round(2)
    dt_imputed['GDP_change_2015_2018'] = ((dt_imputed['GDP_2018'] - dt_imputed['GDP_2015']) / dt_imputed['GDP_2015'] * 100).round(2)

    # Variação percentual das médias PISA
    dt_imputed['PISA_change_2012_2015'] = ((dt_imputed['Mean_2015'] - dt_imputed['Mean_2012']) / dt_imputed['Mean_2012'] * 100).round(2)
    dt_imputed['PISA_change_2015_2018'] = ((dt_imputed['Mean_2018'] - dt_imputed['Mean_2015']) / dt_imputed['Mean_2015'] * 100).round(2)


    #---------------------------------------------------------------------------------------------
    # Parte 6: Rácio GDP / PISA Score por ano
    #---------------------------------------------------------------------------------------------

    dt_imputed['GDP_per_PISA_2012'] = (dt_imputed['GDP_2012'] / dt_imputed['Mean_2012']).round(4)
    dt_imputed['GDP_per_PISA_2015'] = (dt_imputed['GDP_2015'] / dt_imputed['Mean_2015']).round(4)
    dt_imputed['GDP_per_PISA_2018'] = (dt_imputed['GDP_2018'] / dt_imputed['Mean_2018']).round(4)


    #---------------------------------------------------------------------------------------------
    # Parte 7: Diferença absoluta das pontuações PISA e do GDP entre anos
    #---------------------------------------------------------------------------------------------

    # Diferença absoluta
    dt_imputed['GDP_diff_2012_2015'] = (dt_imputed['GDP_2015'] - dt_imputed['GDP_2012']).round(3)
    dt_imputed['GDP_diff_2015_2018'] = (dt_imputed['GDP_2018'] - dt_imputed['GDP_2015']).round(3)

    dt_imputed['PISA_diff_2012_2015'] = (dt_imputed['Mean_2015'] - dt_imputed['Mean_2012']).round(3)
    dt_imputed['PISA_diff_2015_2018'] = (dt_imputed['Mean_2018'] - dt_imputed['Mean_2015']).round(3)

    dt_imputed.to_excel("Dataset_final_pandas.xlsx", index=False)

    return " Resultados gravados em 'Dataset_final_pandas.xlsx' "

In [10]:
run_pandas()


[Pandas] Valores NaN antes: 70
[Pandas] Valores NaN após: 0


" Resultados gravados em 'Dataset_final_pandas.xlsx' "